# PRACTICA 3
## Diego Fernando Valle Morales
## Carné: 20003022

# TEMA KNN


In [1]:
import numpy as np
import imageio as imio
import matplotlib.pyplot as plt
import tensorflow as tf
%reload_ext tensorboard

import tensorboard
tensorboard.__version__

'2.2.2'

In [4]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


'2.2.2'